In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("../../data/raw/environment data - with counties.csv")

In [3]:
relevant_cols = ["NAME", "LATITUDE", "LONGITUDE", "COUNTY", "ELEVATION", "DATE", "CDSD", "EMXP", "PRCP", "CLDD", "DT00", "DT32", "DX32", "DX70", "DX90", "EMNT", "EMXT", "FZF0", "FZF1", "FZF2", "FZF3", "FZF4", "FZF5", "FZF6", "FZF7", "FZF8", "FZF9", "HTDD", "TAVG", "TMAX", "TMIN", "SNOW", "PSUN"]
# consider whether emnt_attribute is needed for the date down the line
target_cols = []

In [4]:
df = data.copy()
relevant = df[relevant_cols].drop_duplicates()
focused = df[target_cols]

# categorize columns
categorical_cols = list(relevant.select_dtypes("object").columns)
#id_cols = [col for col in training.columns if col.endswith("_id")]
id_cols = ["ELEVATION", "DATE"]
target_col = []
exclude_cols = categorical_cols + id_cols + target_col
numerical_cols = [col for col in relevant.columns if col not in exclude_cols]

In [5]:
relevant = relevant.sort_values(by=["COUNTY", "DATE"]).reset_index(drop=True).drop(["NAME", "LATITUDE", "LONGITUDE", "ELEVATION"], axis=1)

In [6]:
display(relevant.tail(6))

,COUNTY,DATE,CDSD,EMXP,PRCP,CLDD,DT00,DT32,DX32,DX70,...,FZF6,FZF7,FZF8,FZF9,HTDD,TAVG,TMAX,TMIN,SNOW,PSUN
1216,Windsor,2019,NaN,2.17,47.30,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1217,Windsor,2019,NaN,2.37,43.76,NaN,19.0,166.0,NaN,NaN,...,28.0,24.0,19.0,15.0,7680.0,NaN,33.0,NaN,NaN,58.6
1218,Windsor,2019,310.0,NaN,NaN,310.0,29.0,176.0,62.0,109.0,...,27.0,23.0,19.0,14.0,8262.0,53.8,32.2,NaN,43.0,NaN
1219,Windsor,2019,359.0,2.47,49.14,359.0,29.0,173.0,57.0,121.0,...,27.0,22.0,20.0,16.0,7840.0,55.7,32.2,NaN,43.9,NaN
1220,Windsor,2019,NaN,2.33,54.18,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1221,Windsor,2019,391.0,1.45,37.19,391.0,12.0,168.0,57.0,125.0,...,26.0,22.0,19.0,15.0,7625.0,55.6,34.5,280.0,45.0,NaN


In [10]:
relevant_groupby = relevant.groupby(["COUNTY", "DATE"])

for county in relevant["COUNTY"].unique():
    county_data = relevant[relevant["COUNTY"] == county]
    for year in county_data["DATE"].unique():
        # get current slice of data from that year for that county
        current_group = relevant_groupby.get_group((county, year))
        # get the names of the columns that are missing all their data
        missing_cols = current_group.loc[:, current_group.isna().sum() == current_group.shape[0]].columns
        # get the names of the columns that have some missing data but not all
        other_cols = [col for col in current_group.columns if col not in missing_cols]
        # fill in the missing values in the columns in other_cols with their respective medians
        current_group[other_cols] = current_group[other_cols].fillna(current_group[other_cols].median())
        # save values in current group back into original dataframe ('relevant')
        relevant.update(current_group, overwrite=False)

D:\Anaconda\envs\Capstone\lib\site-packages\pandas\core\frame.py:2960: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [15]:
display(relevant.head(25))
display(relevant.isna().any())

,COUNTY,DATE,CDSD,EMXP,PRCP,CLDD,DT00,DT32,DX32,DX70,...,FZF6,FZF7,FZF8,FZF9,HTDD,TAVG,TMAX,TMIN,SNOW,PSUN
0,Addison,1990,NaN,2.52,54.870,NaN,2.0,136.0,55.0,107.0,...,24.0,24.0,17.0,11.0,7198.0,NaN,54.20,38.50,131.3,NaN
1,Addison,1990,NaN,2.60,48.940,NaN,2.0,136.0,55.0,107.0,...,24.0,24.0,17.0,11.0,7198.0,NaN,54.20,38.50,53.2,NaN
2,Addison,1990,NaN,2.40,46.650,NaN,2.0,136.0,55.0,107.0,...,24.0,24.0,17.0,11.0,7198.0,NaN,54.20,38.50,69.4,NaN
3,Addison,1991,NaN,1.90,37.150,NaN,30.0,161.0,63.0,109.0,...,27.0,20.0,20.0,11.0,NaN,42.9,53.60,32.30,89.4,NaN
4,Addison,1991,NaN,1.90,37.150,NaN,30.0,161.0,63.0,109.0,...,27.0,20.0,20.0,11.0,NaN,42.9,53.60,32.30,26.0,NaN
5,Addison,1992,66.0,1.66,34.760,66.0,42.0,190.0,75.0,84.0,...,27.0,24.0,15.0,15.0,8685.0,39.5,50.30,28.70,106.6,NaN
6,Addison,1992,269.0,1.67,34.510,269.0,28.0,174.0,41.0,127.0,...,26.0,24.0,13.0,13.0,7909.5,43.9,56.30,31.40,22.3,NaN
7,Addison,1992,209.0,1.41,33.850,209.0,12.0,146.0,58.0,105.0,...,24.0,24.0,20.0,15.0,7134.0,44.4,53.40,35.50,22.5,NaN
8,Addison,1993,142.0,1.85,43.760,142.0,47.0,203.0,88.0,89.0,...,28.0,22.0,20.0,9.0,9145.0,38.5,49.50,27.50,174.0,NaN
9,Addison,1993,387.0,2.00,34.720,387.0,37.0,181.0,53.0,122.0,...,28.0,21.0,19.0,10.0,7726.0,44.0,56.70,31.30,85.9,NaN


COUNTY    False
DATE      False
CDSD       True
EMXP       True
PRCP       True
CLDD       True
DT00       True
DT32       True
DX32       True
DX70       True
DX90       True
EMNT       True
EMXT       True
FZF0       True
FZF1       True
FZF2       True
FZF3       True
FZF4       True
FZF5       True
FZF6       True
FZF7       True
FZF8       True
FZF9       True
HTDD       True
TAVG       True
TMAX       True
TMIN       True
SNOW       True
PSUN       True
dtype: bool

In [16]:
display(relevant.interpolate(method="linear", limit_direction="forward").head(25))
#relevant.head(25)

,COUNTY,DATE,CDSD,EMXP,PRCP,CLDD,DT00,DT32,DX32,DX70,...,FZF6,FZF7,FZF8,FZF9,HTDD,TAVG,TMAX,TMIN,SNOW,PSUN
0,Addison,1990,NaN,2.52,54.870,NaN,2.0,136.0,55.0,107.0,...,24.0,24.0,17.0,11.0,7198.000000,NaN,54.20,38.50,131.3,NaN
1,Addison,1990,NaN,2.60,48.940,NaN,2.0,136.0,55.0,107.0,...,24.0,24.0,17.0,11.0,7198.000000,NaN,54.20,38.50,53.2,NaN
2,Addison,1990,NaN,2.40,46.650,NaN,2.0,136.0,55.0,107.0,...,24.0,24.0,17.0,11.0,7198.000000,NaN,54.20,38.50,69.4,NaN
3,Addison,1991,NaN,1.90,37.150,NaN,30.0,161.0,63.0,109.0,...,27.0,20.0,20.0,11.0,7693.666667,42.9,53.60,32.30,89.4,NaN
4,Addison,1991,NaN,1.90,37.150,NaN,30.0,161.0,63.0,109.0,...,27.0,20.0,20.0,11.0,8189.333333,42.9,53.60,32.30,26.0,NaN
5,Addison,1992,66.0,1.66,34.760,66.0,42.0,190.0,75.0,84.0,...,27.0,24.0,15.0,15.0,8685.000000,39.5,50.30,28.70,106.6,NaN
6,Addison,1992,269.0,1.67,34.510,269.0,28.0,174.0,41.0,127.0,...,26.0,24.0,13.0,13.0,7909.500000,43.9,56.30,31.40,22.3,NaN
7,Addison,1992,209.0,1.41,33.850,209.0,12.0,146.0,58.0,105.0,...,24.0,24.0,20.0,15.0,7134.000000,44.4,53.40,35.50,22.5,NaN
8,Addison,1993,142.0,1.85,43.760,142.0,47.0,203.0,88.0,89.0,...,28.0,22.0,20.0,9.0,9145.000000,38.5,49.50,27.50,174.0,NaN
9,Addison,1993,387.0,2.00,34.720,387.0,37.0,181.0,53.0,122.0,...,28.0,21.0,19.0,10.0,7726.000000,44.0,56.70,31.30,85.9,NaN


In [9]:
tst_grpby = relevant.groupby(["COUNTY", "DATE"])

test1 = tst_grpby.get_group(("Addison", 1990))
#display(test1.isna().sum().unique())
missing_val_cols = test1.loc[:, test1.isna().sum() == test1.shape[0]].columns
other_cols = [col for col in test1.columns if col not in missing_val_cols]
display(missing_val_cols, other_cols)
#display(other_cols)
#test1 = test1.loc[:, test1.isna().sum() == test1.shape[0]]
#display(test1)
#test1[other_cols] = test1[other_cols].fillna(test1[other_cols].median())

Index(['CDSD', 'CLDD', 'TAVG', 'PSUN'], dtype='object')

['COUNTY',
 'DATE',
 'EMXP',
 'PRCP',
 'DT00',
 'DT32',
 'DX32',
 'DX70',
 'DX90',
 'EMNT',
 'EMXT',
 'FZF0',
 'FZF1',
 'FZF2',
 'FZF3',
 'FZF4',
 'FZF5',
 'FZF6',
 'FZF7',
 'FZF8',
 'FZF9',
 'HTDD',
 'TMAX',
 'TMIN',
 'SNOW']

In [ ]:
test3 = relevant.copy()
test3

In [ ]:
test3.update(test1)
test3

In [ ]:
test = relevant[(relevant["DATE"] == 2009) & (relevant["COUNTY"] == "Addison")].drop_duplicates()
#test = test.loc[:, test.isna().sum() == 4]
test
# 4 vals per year
# - missing 1: fill with median of that county at that year
# - missing 2: fill with median of that county at that year
# - missing 3: fill with median of that county at that year
# - missing all: fill with median of entire column after all other years/counties have been filled in
# median over mean:
# - the median is less affected by outliers and skewed data than the mean

In [ ]:
test.equals(test1)

In [ ]:
test_col = "EMXP"
display(test[test_col].mean())
display(test[test_col].median())
test

In [ ]:
#test[test_col].interpolate(method="linear", limit_direction="forward")
test[test_col].fillna(test[test_col].median(), inplace=True)

In [ ]:
test["HTDD"].mean()